In [ ]:
import re

def display_as_markdown(text_string):
    """
    Takes a string containing markdown formatting and displays it as formatted markdown.
    Safely processes escape sequences while preserving LaTeX commands.
    
    Args:
        text_string (str): String containing markdown syntax with \n for newlines
    
    Returns:
        str: The processed markdown string ready for display
    """
    # Use regex to only replace \n when it's not part of a LaTeX command
    # This preserves \neq, \nabla, \norm, etc.
    processed_text = re.sub(r'\\n(?![a-zA-Z])', '\n', text_string)
    
    # Process other common escape sequences safely
    processed_text = re.sub(r'\\t(?![a-zA-Z])', '\t', processed_text)
    processed_text = re.sub(r'\\r(?![a-zA-Z])', '\r', processed_text)
    
    return processed_text


def print_markdown(text_string):
    """
    Prints the string as formatted markdown to console.
    
    Args:
        text_string (str): String containing markdown syntax
    """
    markdown_text = display_as_markdown(text_string)
    print(markdown_text)

In [ ]:
import requests
def call_api(prompt: str, model: str="gpt-4o-mini", temperature: float=0.8) -> str:
    """
    Call the LLM API and retrieve response with probability information.

    This method extends the base API call to include log probabilities and
    entropy calculations for uncertainty quantification.

    Args:
        prompt: The prompt to send to the LLM

    Returns:
        Tuple of (response_content, sentence_entropy, sentence_log_probability)
    """
    api_key = "openai_api_key"  # Replace with your actual API key
    api_url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
    }

    try:
        # Make API request with timeout
        response = requests.post(
            api_url, headers=headers, json=data, timeout=60
        )
        response.raise_for_status()
        response_json = response.json()

        # Extract response content and probability statistics
        raw_response = response_json["choices"][0]["message"]["content"].strip()
        return raw_response
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
        return ""
    except KeyError as e:
        print(f"Unexpected response format: {e}")
        return ""
    except Exception as e:
        print(f"Unexpected error: {e}")
        return ""

In [ ]:
problem_description = """Let's consider a scenario where we have a set of interconnected computer servers intended to distribute data packets. The servers are set up in a network of 6 nodes, ranging from Node 0, the source, to Node 5, the destination. The connections between servers have a certain data transfer capacity, representing the maximum number of data packets that can be sent per unit time.\n\nHere's how the capacities of each connection, represented in thousands of data packets per unit time, are distributed:\n\n- From Node 0 (Source): Can send data to Node 1 (11 packets), Node 2 (8 packets), Node 3 (11 packets), Node 4 (20 packets), and Node 5 (16 packets).\n- From Node 1: Can send data to Node 0 (15 packets), Node 2 (11 packets), Node 3 (16 packets), and Node 5 (20 packets).\n- From Node 2: Can send data to Node 0 (7 packets), Node 1 (4 packets), Node 3 (4 packets), and Node 4 (11 packets).\n- From Node 3: Can send data to Node 0 (4 packets), Node 1 (8 packets), Node 2 (14 packets), and Node 4 (19 packets).\n- From Node 4: Can send data to Node 0 (10 packets), Node 2 (6 packets), Node 3 (19 packets), and Node 5 (13 packets).\n- From Node 5 (Destination): Can send data back to Node 0 (10 packets), Node 1 (15 packets), Node 2 (20 packets), Node 3 (19 packets), and Node 4 (10 packets).\n\nIn this network, a capacity of \"0\" indicates there is no direct connection between the two nodes.\n\nThe task is to calculate the maximum number of data packets that can be sent from Node 0 (the source) to Node 5 (the destination) without exceeding the capacity of any individual connection. In other words, what is the maximum flow possible in this network? What is the maximum data that can be sent from the source to the destination in thousands of packets per unit time?"""

In [ ]:
prompt = f"""You are given a natural language description of an optimization problem.

[Problem Description]: {problem_description}

Your task:
1. **Identify the problem type** (e.g., traveling salesman, production planning, network flow, assignment).

Return the problem type with one sentence introduction.
"""

problem_type = call_api(prompt)
print_markdown(problem_type)

In [ ]:
prompt=f"""You are an optimization expert specializing in mathematical programming. Analyze the following optimization problem: {problem_type}

Your task is to provide a comprehensive modeling analysis with the following structure:

1. Highlight the most important modeling challenge for this specific problem type.
2. Summarize the core modeling logic for a mathematical formulation, addressing the identified challenges. 
   Include the decision variables, objective function, and key constraints.

## Example Analysis Format:

**Problem**: Traveling Salesman Problem
**Primary Challenge**: Subtour elimination - preventing disconnected cycles in the solution
**Core Modeling Logic**:

1. **Decision Variables**
   - Define binary decision variables x[i, j] indicating whether the path goes directly from city i to city j.  
   - Define auxiliary integer variables u[i] for subtour elimination (Miller–Tucker–Zemlin formulation).  

2. **Constraints**
   - Each city must be visited exactly once (in-degree = 1, out-degree = 1).  
   - Subtour elimination: u[i] - u[j] + n * x[i, j] <= n - 1 for all i != j, i != start, j != start.  
   - Fix starting point: choose one city (e.g., 'A') and set u['A'] = 0.  
   - Bounds: u[i] in [1, n-1] for all i != start.  

3. **Objective**
   - Minimize the total travel cost.
   
"""
problem_instruction = call_api(prompt)
print_markdown(problem_instruction)

In [ ]:
prompt = f"""
You are an optimization expert specializing in mathematical programming.

[Problem Text]: {problem_description}

{problem_type}

Your task is to provide **Sets** and **Parameters** for modeling it as a mathematical formulation.

## Part 1: Sets
List all sets needed for this optimization problem. Return as a Python list of dictionaries with the following structure:
- For each set, include 'name' and 'description'.

## Part 2: Parameters  
List all parameters needed for this optimization problem with their actual numerical values. Return as a Python list of dictionaries with the following structure:
- For indexed parameters, include 'name', 'description', 'index_set', and 'values' keys (where 'values' contains the actual numerical data).
- For scalar parameters, include 'name', 'description', and 'value' keys (where 'value' contains the actual number).

IMPORTANT: 
- Do not use markdown formatting or code blocks in your response
- Don't include any other explanations or text
"""
sets_parameters = call_api(prompt)
print_markdown(sets_parameters)

In [ ]:
prompt = f"""
You are an optimization expert specializing in mathematical programming. Given a problem with defined Sets and Parameters, complete the mathematical formulation by defining the decision variables, objective function, and constraints.

[Problem Description]: {problem_description}

[Modeling Instructions]: {problem_instruction}

[Given Sets and Parameters]: {sets_parameters}

Your task is to provide the remaining components for the mathematical formulation:

## Part 3: Decision Variables
Return as a Python list of dictionaries with the following structure:
- For each decision variable, include 'name', 'description', 'index_set', 'domain', and 'interpretation' keys.
- Use the sets defined in the given Sets and Parameters section.

## Part 4: Objective Function
Return as a Python dictionary with the following structure:
- Include 'type' (minimize/maximize), 'expression', and 'description' keys.
- Use mathematical notation referencing the defined sets, parameters, and decision variables.

## Part 5: Constraints
Return as a Python list of dictionaries with the following structure:
- For each constraint type, include 'name', 'mathematical_form', 'description', and 'constraint_type' keys.
- Use proper mathematical notation with summation symbols, indices, and set notation.

IMPORTANT: 
- Do not use markdown formatting or code blocks in your response
- Don't include any other explanations or text
"""
formulation = call_api(prompt)
print_markdown(formulation)

In [ ]:
prompt = f"""You are given a mathematical formulation of an optimization problem. Your task is to generate complete, executable GurobiPy code based on the formulation. 

[Problem Formulation]: {sets_parameters}, {formulation}
"""

prompt += """Please generate Python code that uses the GurobiPy library to solve the problem. """

prompt += """
The code must:
1. Use `from gurobipy import *` as the import command.
2. Create a model using `Model()`.
3. Define all sets, parameters, variables, objective function, and constraints exactly as described in the formulation. 
4. **CRITICAL - Avoid Index Errors**: 
- Only create variables for valid index combinations that exist in your defined sets/parameters
- When using summation in constraints or objective, always check if the index combination exists before referencing it
- Use conditional logic like `if (i,j) in valid_pairs` or `if parameter[i][j] > 0` before accessing variables
- For network problems, only reference edges that actually exist in the network
- **Do not use chained inequalities like `a <= x <= b`; instead, add separate constraints for lower and upper bounds or use `addRange`.**
5. Call `.optimize()` to solve the model.
6. After solving, include output logic that does the following:
- If the model is solved to optimality:
   - Print: `"Optimal solution found"`
   - Print: `"Objective Value: <value>"`
   - Print: `"BEGIN_VARIABLES"`
   - For each decision variable (including indexed variables), print:
   `"Variable <var.name>: <value>"`
   - Print: `"END_VARIABLES"`
- If the model is infeasible or unbounded:
   - Print: `"Model is infeasible or unbounded"`

Do not include any explanations or comments. Do not use markdown formatting or code blocks. Return only the raw Python code as a string that can be executed directly.
"""
code = call_api(prompt)

In [ ]:
from feedback import execute_optimization_code
if code.startswith("```") and "```" in code[3:]:
    # Find the first and last triple backticks
    first_marker = code.find("```")
    # Skip the language identifier if present
    lang_end = code.find("\n", first_marker)
    # Find the closing backticks
    second_marker = code.find("```", lang_end)
    if second_marker > first_marker:
        # Extract the code without the backticks and language identifier
        code = code[lang_end + 1 : second_marker].strip()
# remove the leading/trailing double quotes that are part of the content
if code.startswith('"') and code.endswith('"'):
    code = code[1:-1]

# turn \n and \" into real newlines/quotes
code = code.encode().decode("unicode_escape")
result = execute_optimization_code(code)
print(result)

In [ ]:
code

In [ ]:
prompt = f"""You are an optimization expert. 
Check the provided solving code against the problem description. 
Identify any errors or inconsistencies strictly in terms of:
1. Numerical Parameter Consistency (e.g., values used in code vs problem text).
2. Logical correctness (e.g., conservation laws such as inflow = outflow, balance equations, feasibility of constraints).

Do not comment on style, formatting, or efficiency. Report only true mismatches or logical violations. 

[Problem Description]: {problem_description} 
[Code]: {code}"""
inconsistency = call_api(prompt)
print_markdown(inconsistency)